In [1]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

d:\Programming\DSMP 2.0\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [3]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters - You have to mention the search space inside your optimization function
    n_estimators = trial.suggest_int('n_estimators', 50, 200) # This siggest_int function communating the sampler function and that sampler function giving me the next value for trying
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize

In [ ]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy | Default sampler = TPE sampler
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

# By calling the optimize function from study we got trial object - which is passed in the objective function

[I 2025-06-01 17:10:09,153] A new study created in memory with name: no-name-b5b08dc8-75cc-40d1-9c83-c3a3703e19cc
[I 2025-06-01 17:10:09,582] Trial 0 finished with value: 0.7541899441340781 and parameters: {'n_estimators': 100, 'max_depth': 9}. Best is trial 0 with value: 0.7541899441340781.
[I 2025-06-01 17:10:10,229] Trial 1 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 182, 'max_depth': 13}. Best is trial 1 with value: 0.7728119180633147.
[I 2025-06-01 17:10:10,455] Trial 2 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 69, 'max_depth': 5}. Best is trial 2 with value: 0.7746741154562384.
[I 2025-06-01 17:10:10,709] Trial 3 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 65, 'max_depth': 8}. Best is trial 2 with value: 0.7746741154562384.
[I 2025-06-01 17:10:11,157] Trial 4 finished with value: 0.7858472998137803 and parameters: {'n_estimators': 121, 'max_depth': 15}. Best is trial 4 with value: 0.785847299

In [6]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7858472998137803
Best hyperparameters: {'n_estimators': 121, 'max_depth': 15}


In [7]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.76


## Samplers in Optuna

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize

In [9]:
# Random Sampler - Select the next parameter value randomly
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-06-01 17:19:47,227] A new study created in memory with name: no-name-d3b0628a-1d89-4c20-abaf-32fa3d05d7fc
[I 2025-06-01 17:19:47,459] Trial 0 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 57, 'max_depth': 13}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-06-01 17:19:47,667] Trial 1 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 63, 'max_depth': 4}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-06-01 17:19:48,207] Trial 2 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 147, 'max_depth': 19}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-06-01 17:19:48,834] Trial 3 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 187, 'max_depth': 7}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-06-01 17:19:49,423] Trial 4 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 162, 'max_depth': 19}. Best is trial 0 with value: 0.77281191

In [10]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 195, 'max_depth': 18}


In [11]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [13]:
# If you want to use "Grid Search Sampler" you have mention your search outside the objective function and have to pass in the sampler (Below code)
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [ ]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective) # Here you dont pass number to trials because it will try every single combination of hyperparameters in your search space

[I 2025-06-01 17:22:32,524] A new study created in memory with name: no-name-2b347405-6496-41db-b02f-07d1ea625c2f
[I 2025-06-01 17:22:32,890] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-06-01 17:22:33,371] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-06-01 17:22:33,552] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-06-01 17:22:33,887] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-06-01 17:22:34,208] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.74


In [5]:
# Create a study and optimize it using CmaEsSampler - !pip install cmaes
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.CmaEsSampler())
study.optimize(objective, n_trials=50)

[I 2025-06-01 17:47:52,764] A new study created in memory with name: no-name-8b82da43-45f7-452b-9678-6c6165c93d2b
[I 2025-06-01 17:47:53,138] Trial 0 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 108, 'max_depth': 10}. Best is trial 0 with value: 0.7597765363128491.
[I 2025-06-01 17:47:53,541] Trial 1 finished with value: 0.7728119180633146 and parameters: {'n_estimators': 122, 'max_depth': 12}. Best is trial 1 with value: 0.7728119180633146.
[I 2025-06-01 17:47:53,888] Trial 2 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 105, 'max_depth': 9}. Best is trial 1 with value: 0.7728119180633146.
[I 2025-06-01 17:47:54,298] Trial 3 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 118, 'max_depth': 18}. Best is trial 3 with value: 0.7765363128491621.
[I 2025-06-01 17:47:54,644] Trial 4 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 102, 'max_depth': 14}. Best is trial 3 with value: 0.77653

In [6]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 104, 'max_depth': 12}


In [7]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.76


In [8]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_max_depth,params_n_estimators,system_attrs_cma:generation,system_attrs_cma:large_n_eval,system_attrs_cma:n_restarts,system_attrs_cma:n_restarts_with_large,system_attrs_cma:optimizer:0,system_attrs_cma:optimizer:1,system_attrs_cma:popsize,system_attrs_cma:poptype,system_attrs_cma:small_n_eval,state
0,0,0.759777,2025-06-01 17:47:52.765276,2025-06-01 17:47:53.138253,0 days 00:00:00.372977,10,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.772812,2025-06-01 17:47:53.139061,2025-06-01 17:47:53.541833,0 days 00:00:00.402772,12,122,0.0,0.0,0.0,0.0,NaN,NaN,6.0,small,0.0,COMPLETE
2,2,0.761639,2025-06-01 17:47:53.542619,2025-06-01 17:47:53.888252,0 days 00:00:00.345633,9,105,0.0,0.0,0.0,0.0,NaN,NaN,6.0,small,0.0,COMPLETE
3,3,0.776536,2025-06-01 17:47:53.889057,2025-06-01 17:47:54.298194,0 days 00:00:00.409137,18,118,0.0,0.0,0.0,0.0,NaN,NaN,6.0,small,0.0,COMPLETE
4,4,0.763501,2025-06-01 17:47:54.299021,2025-06-01 17:47:54.644789,0 days 00:00:00.345768,14,102,0.0,0.0,0.0,0.0,NaN,NaN,6.0,small,0.0,COMPLETE
5,5,0.763501,2025-06-01 17:47:54.645607,2025-06-01 17:47:55.032142,0 days 00:00:00.386535,10,105,0.0,0.0,0.0,0.0,NaN,NaN,6.0,small,0.0,COMPLETE
6,6,0.767225,2025-06-01 17:47:55.033001,2025-06-01 17:47:55.287762,0 days 00:00:00.254761,7,82,0.0,0.0,0.0,0.0,NaN,NaN,6.0,small,0.0,COMPLETE
7,7,0.767225,2025-06-01 17:47:55.288526,2025-06-01 17:47:55.722536,0 days 00:00:00.434010,14,133,1.0,0.0,0.0,0.0,8004954e060000000000008c0a636d6165732e5f636d61...,3200000000000000000000000000000000000000000000...,6.0,small,0.0,COMPLETE
8,8,0.765363,2025-06-01 17:47:55.723318,2025-06-01 17:47:56.040564,0 days 00:00:00.317246,11,98,1.0,0.0,0.0,0.0,NaN,NaN,6.0,small,0.0,COMPLETE
9,9,0.778399,2025-06-01 17:47:56.041367,2025-06-01 17:47:56.410510,0 days 00:00:00.369143,17,113,1.0,0.0,0.0,0.0,NaN,NaN,6.0,small,0.0,COMPLETE


In [9]:
study.sampler

## Optuna Visualizations

In [10]:
# For visualizations
import plotly.express as px
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [11]:
# 1. Optimization History
plot_optimization_history(study).show()

In [12]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [13]:
# 3. Slice Plot
plot_slice(study).show()

In [14]:
# 4. Contour Plot
plot_contour(study).show()

In [15]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [17]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [16]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [18]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-06-01 17:55:28,059] A new study created in memory with name: no-name-c24d9c0f-0ca8-41ad-8d2f-fdf9223dc7c7
[I 2025-06-01 17:55:29,409] Trial 0 finished with value: 0.7486033519553073 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 177, 'learning_rate': 0.016720016656443928, 'max_depth': 6, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.7486033519553073.
[I 2025-06-01 17:55:30,353] Trial 1 finished with value: 0.7635009310986964 and parameters: {'classifier': 'RandomForest', 'n_estimators': 282, 'max_depth': 9, 'min_samples_split': 2, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 1 with value: 0.7635009310986964.
[I 2025-06-01 17:55:30,523] Trial 2 finished with value: 0.7709497206703911 and parameters: {'classifier': 'RandomForest', 'n_estimators': 60, 'max_depth': 13, 'min_samples_split': 9, 'min_samples_leaf': 8, 'bootstrap': False}. Best is trial 2 with value: 0.7709497206703911.
[I 2025-06-01 17:55:30,705] Trial 3 

In [19]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.14978949624524346, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [20]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.748603,2025-06-01 17:55:28.061334,2025-06-01 17:55:29.409388,0 days 00:00:01.348054,NaN,NaN,GradientBoosting,NaN,NaN,0.016720,6.0,3.0,6.0,177.0,COMPLETE
1,1,0.763501,2025-06-01 17:55:29.410241,2025-06-01 17:55:30.353249,0 days 00:00:00.943008,NaN,True,RandomForest,NaN,NaN,NaN,9.0,4.0,2.0,282.0,COMPLETE
2,2,0.770950,2025-06-01 17:55:30.354134,2025-06-01 17:55:30.523086,0 days 00:00:00.168952,NaN,False,RandomForest,NaN,NaN,NaN,13.0,8.0,9.0,60.0,COMPLETE
3,3,0.765363,2025-06-01 17:55:30.523966,2025-06-01 17:55:30.705198,0 days 00:00:00.181232,NaN,False,RandomForest,NaN,NaN,NaN,11.0,6.0,4.0,62.0,COMPLETE
4,4,0.728119,2025-06-01 17:55:30.706051,2025-06-01 17:55:32.877594,0 days 00:00:02.171543,NaN,NaN,GradientBoosting,NaN,NaN,0.042239,18.0,3.0,9.0,187.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.782123,2025-06-01 17:55:50.045763,2025-06-01 17:55:50.066200,0 days 00:00:00.020437,0.382501,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.785847,2025-06-01 17:55:50.066931,2025-06-01 17:55:50.086198,0 days 00:00:00.019267,0.218164,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.787709,2025-06-01 17:55:50.086946,2025-06-01 17:55:50.104779,0 days 00:00:00.017833,0.101940,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.789572,2025-06-01 17:55:50.105518,2025-06-01 17:55:50.127627,0 days 00:00:00.022109,0.127061,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [21]:
study.trials_dataframe()['params_classifier'].value_counts()

params_classifier
SVM                 75
RandomForest        15
GradientBoosting    10
Name: count, dtype: int64

In [22]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.747114
RandomForest        0.769088
SVM                 0.777107
Name: value, dtype: float64

In [23]:
# 1. Optimization History
plot_optimization_history(study).show()

In [24]:
# 3. Slice Plot
plot_slice(study).show()

In [25]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [28]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")

[I 2025-06-01 18:04:27,543] A new study created in memory with name: no-name-229ad72c-fa0c-4741-a5b8-a90c40914458


[0]	train-mlogloss:0.96180	eval-mlogloss:0.95571
[1]	train-mlogloss:0.90844	eval-mlogloss:0.89620
[2]	train-mlogloss:0.87709	eval-mlogloss:0.87505
[3]	train-mlogloss:0.75527	eval-mlogloss:0.73911
[4]	train-mlogloss:0.72378	eval-mlogloss:0.70088
[5]	train-mlogloss:0.66625	eval-mlogloss:0.63672
[6]	train-mlogloss:0.61152	eval-mlogloss:0.57373
[7]	train-mlogloss:0.56567	eval-mlogloss:0.52396
[8]	train-mlogloss:0.54005	eval-mlogloss:0.49774
[9]	train-mlogloss:0.50174	eval-mlogloss:0.45602
[10]	train-mlogloss:0.47205	eval-mlogloss:0.42248
[11]	train-mlogloss:0.46674	eval-mlogloss:0.41270
[12]	train-mlogloss:0.44206	eval-mlogloss:0.38676
[13]	train-mlogloss:0.44130	eval-mlogloss:0.38580
[14]	train-mlogloss:0.42933	eval-mlogloss:0.37384
[15]	train-mlogloss:0.42759	eval-mlogloss:0.37121
[16]	train-mlogloss:0.42626	eval-mlogloss:0.36901
[17]	train-mlogloss:0.42451	eval-mlogloss:0.36466
[18]	train-mlogloss:0.42089	eval-mlogloss:0.35789
[19]	train-mlogloss:0.42008	eval-mlogloss:0.35729
[20]	train

[I 2025-06-01 18:04:28,117] Trial 0 finished with value: 0.9666666666666667 and parameters: {'lambda': 1.2792951323824154e-06, 'alpha': 0.0001362407565213009, 'eta': 0.1570356083797959, 'gamma': 5.102265774779156e-08, 'max_depth': 5, 'min_child_weight': 10, 'subsample': 0.5841532615111458, 'colsample_bytree': 0.4856153033253831}. Best is trial 0 with value: 0.9666666666666667.


[0]	train-mlogloss:0.87614	eval-mlogloss:0.86862
[1]	train-mlogloss:0.71239	eval-mlogloss:0.69128
[2]	train-mlogloss:0.59343	eval-mlogloss:0.56897
[3]	train-mlogloss:0.49936	eval-mlogloss:0.46631
[4]	train-mlogloss:0.44428	eval-mlogloss:0.40805
[5]	train-mlogloss:0.39623	eval-mlogloss:0.35377
[6]	train-mlogloss:0.35899	eval-mlogloss:0.31253
[7]	train-mlogloss:0.34611	eval-mlogloss:0.30062
[8]	train-mlogloss:0.31901	eval-mlogloss:0.26601
[9]	train-mlogloss:0.30775	eval-mlogloss:0.24811
[10]	train-mlogloss:0.29819	eval-mlogloss:0.23675
[11]	train-mlogloss:0.29721	eval-mlogloss:0.23426
[12]	train-mlogloss:0.28411	eval-mlogloss:0.22122
[13]	train-mlogloss:0.28363	eval-mlogloss:0.22088
[14]	train-mlogloss:0.28317	eval-mlogloss:0.22069
[15]	train-mlogloss:0.28311	eval-mlogloss:0.22029
[16]	train-mlogloss:0.28287	eval-mlogloss:0.21990
[17]	train-mlogloss:0.28284	eval-mlogloss:0.21995
[18]	train-mlogloss:0.28272	eval-mlogloss:0.21971
[19]	train-mlogloss:0.28221	eval-mlogloss:0.21954
[20]	train

[I 2025-06-01 18:04:28,733] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.06480498093442934, 'alpha': 4.2564310263769396e-05, 'eta': 0.18746849946922456, 'gamma': 5.847808192884618e-07, 'max_depth': 5, 'min_child_weight': 10, 'subsample': 0.7059113977044869, 'colsample_bytree': 0.8894911713090226}. Best is trial 1 with value: 1.0.


...

[0]	train-mlogloss:1.03518	eval-mlogloss:1.03192
[1]	train-mlogloss:1.00191	eval-mlogloss:1.00137
[2]	train-mlogloss:0.95225	eval-mlogloss:0.94952
[3]	train-mlogloss:0.90038	eval-mlogloss:0.89669
[4]	train-mlogloss:0.85275	eval-mlogloss:0.84763
[5]	train-mlogloss:0.80890	eval-mlogloss:0.80092
[6]	train-mlogloss:0.76832	eval-mlogloss:0.75704
[7]	train-mlogloss:0.73052	eval-mlogloss:0.71758
[8]	train-mlogloss:0.69601	eval-mlogloss:0.68100


[I 2025-06-01 18:04:32,980] Trial 48 pruned. Trial was pruned at iteration 8.


[0]	train-mlogloss:0.83114	eval-mlogloss:0.82711
[1]	train-mlogloss:0.65304	eval-mlogloss:0.63399


[I 2025-06-01 18:04:33,039] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 0.06480498093442934, 'alpha': 4.2564310263769396e-05, 'eta': 0.18746849946922456, 'gamma': 5.847808192884618e-07, 'max_depth': 5, 'min_child_weight': 10, 'subsample': 0.7059113977044869, 'colsample_bytree': 0.8894911713090226}
Best accuracy: 1.0


In [29]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()